## Calculate ATO for TAZ centroid(s)

1. Run 1_setup_network.ipynb to create NetworkDataset_MM
2. Create OD Cost Matrix Layer
3. Add origin(s)
4. Add destinations
5. Solve
6. Join attributes to solved Lines layer
7. Weight HH and JOB by time decay
8. Sum

In [1]:
import arcpy
import os

arcpy.CheckOutExtension("network")

from arcgis.features import SpatialDataFrame
import pandas as pd
# from arcgis.features import GeoAccessor, GeoSeriesAccessor

def survey_weight(t):
    if t <= 3:
        return 1
    elif (t > 3) & (t <= 20):
        return -0.0382 * t + 1.1293
    elif t > 20:
        return 1/(1 + math.exp(0.1092 * t - 1.5604))
    else:
        return 0

# set base path - jupyter or arcgis
try:
    aprx = arcpy.mp.ArcGISProject("CURRENT")
    print("base path must be explicitly set when running in arcgis pro")
    base_path = r"c:\wfrc\ato" # update this
except OSError:
    base_path = "."

# Set to True to limit to ~30 TAZs
testing = False

base_gdb = os.path.join(base_path, "nd.gdb")

In [2]:
# from 1_setup_network.ipynb

if testing:
    centroids = os.path.join(base_path, r"shp\taz_wfrc.gdb\taz_centroids_draper")
else:
    centroids = os.path.join(base_path, r"shp\taz_wfrc.gdb\taz_centroids_snapped")

# baseline
arcpy.env.workspace = base_gdb
input_network_dataset = os.path.join(base_gdb, r"NetworkDataset\NetworkDataset_ND")
output_file = "driving_baseline"


# mod
#target_gdb =  os.path.join(base_path, "mod_drive.gdb")
#arcpy.env.workspace = target_gdb
#input_network_dataset = os.path.join(target_gdb, r"NetworkDataset\NetworkDataset_ND")
#output_file = "driving_mod"

mode = "Driving" # Driving | Transit

In [3]:
# Create OD Cost Matrix Layers
arcpy.na.MakeODCostMatrixAnalysisLayer(
    network_data_source = input_network_dataset, 
    layer_name = "OD Cost Matrix", 
    travel_mode = mode, 
    cutoff = 60.0,
    line_shape = "NO_LINES"
)

<Result 'OD Cost Matrix'>

In [4]:
# add ODs for simple network
arcpy.na.AddLocations(
    "OD Cost Matrix", 
    "Origins",
    centroids, 
    "Name CO_TAZID #;TargetDestinationCount # #;CurbApproach # 0;Cutoff_Length # #;Cutoff_Mins_DriveTime # #", 
    "10000 Meters"
)
arcpy.na.AddLocations(
    "OD Cost Matrix", 
    "Destinations", 
    centroids, 
    "Name CO_TAZID #;CurbApproach # 0", 
    "10000 Meters"
)

<Result 'OD Cost Matrix'>

In [5]:
%%time
# Solve
arcpy.na.Solve("OD Cost Matrix", "SKIP", "TERMINATE", None, '')

Wall time: 5min 48s


id,value
0,a Layer object
1,true


In [6]:
%%time
od = pd.DataFrame.spatial.from_featureclass(r"OD Cost Matrix\Lines")

Wall time: 19 s


In [7]:
od.head()

,ObjectID,Name,OriginID,DestinationID,DestinationRank,Total_Mins_BikeOnly,Total_Mins_BikeTransitOnly,Total_Mins_DriveOnly,Total_Mins_PedOnly,Total_Mins_PedTransitOnly,Total_Mins_TransitOnly,Total_Miles,SHAPE
0,1,491896 - 491896,1,1,1,None,None,0.000000,None,None,None,None,None
1,2,491896 - 491894,1,2814,2,None,None,0.820445,None,None,None,None,None
2,3,491896 - 491910,1,1401,3,None,None,0.902399,None,None,None,None,None
3,4,491896 - 491907,1,2249,4,None,None,1.237324,None,None,None,None,None
4,5,491896 - 491893,1,1390,5,None,None,1.246772,None,None,None,None,None


In [8]:
%%time
od['Origin_CO_TAZID'] = od['Name'].apply(lambda x: int(x.split(' - ')[0]))
od['Dest_CO_TAZID'] = od['Name'].apply(lambda x: int(x.split(' - ')[1]))

Wall time: 8.98 s


In [9]:
taz = pd.DataFrame.spatial.from_featureclass(os.path.join(r"shp\taz_wfrc.gdb", "ATO"))

In [10]:
taz = taz[['CO_TAZID', 'HH_19', 'JOB_19', 'JOBAUTO_19', 'HHAUTO_19', 'JOBTRANSIT_19', 'HHTRANSIT_19']]

In [11]:
df = pd.merge(od, taz, left_on="Dest_CO_TAZID", right_on="CO_TAZID", )

In [12]:
df.head()

,ObjectID,Name,OriginID,DestinationID,DestinationRank,Total_Mins_BikeOnly,Total_Mins_BikeTransitOnly,Total_Mins_DriveOnly,Total_Mins_PedOnly,Total_Mins_PedTransitOnly,...,SHAPE,Origin_CO_TAZID,Dest_CO_TAZID,CO_TAZID,HH_19,JOB_19,JOBAUTO_19,HHAUTO_19,JOBTRANSIT_19,HHTRANSIT_19
0,1,491896 - 491896,1,1,1,None,None,0.000000,None,None,...,None,491896,491896,491896,0.0,286.6,46951,37564,0,0
1,3406,492490 - 491896,2,1,1108,None,None,38.074487,None,None,...,None,492490,491896,491896,0.0,286.6,46951,37564,0,0
2,5680,492493 - 491896,3,1,1108,None,None,39.530488,None,None,...,None,492493,491896,491896,0.0,286.6,46951,37564,0,0
3,7941,492632 - 491896,4,1,1108,None,None,38.741992,None,None,...,None,492632,491896,491896,0.0,286.6,46951,37564,0,0
4,9130,491859 - 491896,5,1,31,None,None,8.572034,None,None,...,None,491859,491896,491896,0.0,286.6,46951,37564,0,0


In [13]:
# Weight outputs

if mode == 'Driving':
    df['travel_time'] = df['Total_Mins_DriveOnly']
elif mode == 'Transit':
    df['travel_time'] = df['Total_Mins_PedTransitOnly']

df['survey_weight'] = df['travel_time'].apply(lambda x: survey_weight(x))
df['survey_weight'] = df['survey_weight'].round(3)

df['weighted_jobs'] = df['survey_weight'] * df['JOB_19']
df['weighted_hh'] = df['survey_weight'] * df['HH_19']
df['weighted_jobs'] = round(df['weighted_jobs'])
df['weighted_hh'] = round(df['weighted_hh'])

df['ato'] = df['weighted_jobs'] + df['weighted_hh']

In [14]:
# write to disk
df[['Name', 'Origin_CO_TAZID', 'Dest_CO_TAZID', 'travel_time', 'survey_weight',
         'weighted_jobs', 'weighted_hh', 'ato']].to_csv(output_file + '.csv')

In [15]:
taz_summary = df.groupby('Origin_CO_TAZID').agg(
    jobs=pd.NamedAgg(column='weighted_jobs', aggfunc=sum),
    hh=pd.NamedAgg(column='weighted_hh', aggfunc=sum)
)
taz_summary['ato'] = taz_summary['jobs'] + taz_summary['hh']
taz_summary.to_csv(output_file + '_summary.csv')